<a href="https://colab.research.google.com/github/descartesmbogning/Marburg-virus-research/blob/main/3_analysis_topics_fund.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [ ]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

In [ ]:
from scipy import stats #the only import you actually need for this
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [ ]:
# function that tests distribution then picks test accordingly
def test_sig(x, y):
    if stats.shapiro(x).pvalue >= 0.05 and stats.shapiro(y).pvalue >= 0.05: #if they are normally distriuted
        print("t-test")
        return stats.ttest_ind(x,y)
    else:
        print("mann-whitney")
        return stats.mannwhitneyu(x, y)

## import data

In [ ]:
data = pd.read_csv('D:\MARBURG VIRUS DISEASES\TABLES/data_fund_region_country_and_org_clean_dummies_col_replace_grp.csv')#.drop(['Addresses'], axis=1)
# data = data.loc[data['funding_yes'] == 1]
data.head(2)

,Unnamed: 0,wos_ID,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,...,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year,year_group
0,0,wos:000904661800001,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"green accepted, gold",NaN,14.0,14.0,2023.0,2018-2023
1,1,wos:000921279800001,1,1,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0,2018-2023


In [ ]:
data.shape

(932, 522)

In [ ]:
data_topics = pd.read_pickle('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/data_prep_lemmatized_topics.pkl')[['Topic N°']].reset_index()
data_topics.head(2)

,wos_ID,Topic N°
0,wos:000904661800001,3
1,wos:000921279800001,5


In [ ]:
data_topics.shape

(932, 2)

In [ ]:
data_and_topics_number = pd.concat([data.set_index('wos_ID'),
                                   data_topics.set_index('wos_ID')],
                                  axis=1)
data_and_topics_number.head(2)

,Unnamed: 0,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,/,...,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year,year_group,Topic N°
wos_ID,,,,,,,,,,,,,,,,,,,,,
wos:000904661800001,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"green accepted, gold",NaN,14.0,14.0,2023.0,2018-2023,3
wos:000921279800001,1,1,1,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0,2018-2023,5


In [ ]:
data_and_topics_number.shape

(932, 522)

# The topics of Marburg Virus Diseases publications

## proportion all

In [ ]:
countries_topics_list_all = data_and_topics_number.reset_index().groupby(['Topic N°']).count().T
# countries_topics_list_all.rename(columns = {0:'all_is_no_oa',
#                               1:'all_is_oa'}, inplace = True)

countries_topics_list_all['all_total'] = countries_topics_list_all[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
countries_topics_list_all.columns = countries_topics_list_all.columns.astype(str)

for col in countries_topics_list_all.columns:
    countries_topics_list_all[col + '_%all'] = countries_topics_list_all[col]/countries_topics_list_all['all_total']*100

# save data
# countries_topics_list_all.to_csv(f'../TABLES/countries_all_oa_list.csv')
countries_topics_list_all

Topic N°,0,1,2,3,4,5,6,7,8,9,...,1_%all,2_%all,3_%all,4_%all,5_%all,6_%all,7_%all,8_%all,9_%all,all_total_%all
wos_ID,66,79,59,79,51,141,91,170,107,89,...,8.476395,6.330472,8.476395,5.472103,15.128755,9.763948,18.240343,11.480687,9.549356,100.0
Unnamed: 0,66,79,59,79,51,141,91,170,107,89,...,8.476395,6.330472,8.476395,5.472103,15.128755,9.763948,18.240343,11.480687,9.549356,100.0
funding_yes,66,79,59,79,51,141,91,170,107,89,...,8.476395,6.330472,8.476395,5.472103,15.128755,9.763948,18.240343,11.480687,9.549356,100.0
open_access_yes,66,79,59,79,51,141,91,170,107,89,...,8.476395,6.330472,8.476395,5.472103,15.128755,9.763948,18.240343,11.480687,9.549356,100.0
Funding Name Preferred,12,54,39,56,44,95,77,87,43,68,...,9.391304,6.782609,9.739130,7.652174,16.521739,13.391304,15.130435,7.478261,11.826087,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Times Cited, All Databases",56,78,55,71,49,136,86,158,96,86,...,8.955224,6.314581,8.151550,5.625718,15.614237,9.873708,18.140069,11.021814,9.873708,100.0
180 Day Usage Count,8,16,22,25,20,31,36,78,40,36,...,5.128205,7.051282,8.012821,6.410256,9.935897,11.538462,25.000000,12.820513,11.538462,100.0
Since 2013 Usage Count,60,77,58,76,50,130,87,167,103,87,...,8.603352,6.480447,8.491620,5.586592,14.525140,9.720670,18.659218,11.508380,9.720670,100.0
Publication Year,65,79,58,79,51,140,91,170,106,88,...,8.522114,6.256742,8.522114,5.501618,15.102481,9.816613,18.338727,11.434736,9.492988,100.0


In [ ]:
# countries_topics_list_all.describe()

## proportion fund

In [ ]:
data_fund = data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 1]
data_fund.shape

(577, 522)

In [ ]:
countries_topics_list_fund = data_fund.reset_index().groupby(['Topic N°']).count().T
# countries_topics_list_fund.rename(columns = {0:'fund_is_no_oa',
#                               1:'fund_is_oa'}, inplace = True)

countries_topics_list_fund['fund_total'] = countries_topics_list_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
countries_topics_list_fund.columns = countries_topics_list_fund.columns.astype(str)

for col in countries_topics_list_fund.columns:
    countries_topics_list_fund[col+'_%fund'] = countries_topics_list_fund[col]/countries_topics_list_fund['fund_total']*100

# save data
# countries_topics_list_fund.to_csv(f'../TABLES/countries_fund_oa_list.csv')

countries_topics_list_fund

Topic N°,0,1,2,3,4,5,6,7,8,9,...,1_%fund,2_%fund,3_%fund,4_%fund,5_%fund,6_%fund,7_%fund,8_%fund,9_%fund,fund_total_%fund
wos_ID,12,55,40,56,44,95,77,87,43,68,...,9.532062,6.932409,9.705373,7.625650,16.464471,13.344887,15.077990,7.452340,11.785095,100.0
Unnamed: 0,12,55,40,56,44,95,77,87,43,68,...,9.532062,6.932409,9.705373,7.625650,16.464471,13.344887,15.077990,7.452340,11.785095,100.0
funding_yes,12,55,40,56,44,95,77,87,43,68,...,9.532062,6.932409,9.705373,7.625650,16.464471,13.344887,15.077990,7.452340,11.785095,100.0
open_access_yes,12,55,40,56,44,95,77,87,43,68,...,9.532062,6.932409,9.705373,7.625650,16.464471,13.344887,15.077990,7.452340,11.785095,100.0
Funding Name Preferred,12,54,39,56,44,95,77,87,43,68,...,9.391304,6.782609,9.739130,7.652174,16.521739,13.391304,15.130435,7.478261,11.826087,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Times Cited, All Databases",12,54,38,51,43,90,72,81,41,65,...,9.872029,6.946984,9.323583,7.861060,16.453382,13.162706,14.808044,7.495430,11.882998,100.0
180 Day Usage Count,5,13,19,17,18,25,33,51,23,32,...,5.508475,8.050847,7.203390,7.627119,10.593220,13.983051,21.610169,9.745763,13.559322,100.0
Since 2013 Usage Count,12,53,40,54,43,89,74,85,43,67,...,9.464286,7.142857,9.642857,7.678571,15.892857,13.214286,15.178571,7.678571,11.964286,100.0
Publication Year,12,55,39,56,44,94,77,87,42,67,...,9.598604,6.806283,9.773124,7.678883,16.404887,13.438045,15.183246,7.329843,11.692845,100.0


In [ ]:
# data_no_fund[9].value_counts()

In [ ]:
countries_topics_list_no_fund['no_fund_total'].head(60)

wos_ID                             355
Unnamed: 0                         355
funding_yes                        355
open_access_yes                    355
Funding Name Preferred               0
europe and central asia              0
north america                        0
africa                               0
sub-saharan africa                   0
east asia and pacific                0
/                                    0
middle east and north africa         0
south asia                           0
latin america and the caribbean      0
multilateral                         0
germany                              0
belgium                              0
finland                              0
usa                                  0
african union                        0
nigeria                              0
france                               0
japan                                0
united kingdom                       0
european union                       0
austria                  

## proportion no fund

In [ ]:
data_no_fund = data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 0]
data_no_fund.head(2)

,Unnamed: 0,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,/,...,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year,year_group,Topic N°
wos_ID,,,,,,,,,,,,,,,,,,,,,
wos:000904661800001,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"green accepted, gold",NaN,14.0,14.0,2023.0,2018-2023,3
wos:000849602200001,4,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,gold,2.0,14.0,14.0,2022.0,2018-2023,8


In [ ]:
countries_topics_list_no_fund = data_no_fund.reset_index().groupby(['Topic N°']).count().T
# countries_topics_list_no_fund.rename(columns = {0:'no_fund_is_no_oa',
#                               1:'no_fund_is_oa'}, inplace = True)

countries_topics_list_no_fund['no_fund_total'] = countries_topics_list_no_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
countries_topics_list_no_fund.columns = countries_topics_list_no_fund.columns.astype(str)

for col in countries_topics_list_no_fund.columns:
    countries_topics_list_no_fund[col + '_%no_fund'] = countries_topics_list_no_fund[col]/countries_topics_list_no_fund['no_fund_total']*100

# countries_topics_list_no_fund['%_diff(%no_fund_is_oa-%no_fund_is_no_oa)'] = countries_topics_list_no_fund['%no_fund_is_oa'] - countries_topics_list_no_fund['%no_fund_is_no_oa']

# save data
# countries_topics_list_no_fund.to_csv(f'../TABLES/countries_fund_no_oa_list.csv')

countries_topics_list_no_fund

Topic N°,0,1,2,3,4,5,6,7,8,9,...,1_%no_fund,2_%no_fund,3_%no_fund,4_%no_fund,5_%no_fund,6_%no_fund,7_%no_fund,8_%no_fund,9_%no_fund,no_fund_total_%no_fund
wos_ID,54,24,19,23,7,46,14,83,64,21,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
Unnamed: 0,54,24,19,23,7,46,14,83,64,21,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
funding_yes,54,24,19,23,7,46,14,83,64,21,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
open_access_yes,54,24,19,23,7,46,14,83,64,21,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
Funding Name Preferred,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Times Cited, All Databases",44,24,17,20,6,46,14,77,55,21,...,7.407407,5.246914,6.172840,1.851852,14.197531,4.320988,23.765432,16.975309,6.481481,100.0
180 Day Usage Count,3,3,3,8,2,6,3,27,17,4,...,3.947368,3.947368,10.526316,2.631579,7.894737,3.947368,35.526316,22.368421,5.263158,100.0
Since 2013 Usage Count,48,24,18,22,7,41,13,82,60,20,...,7.164179,5.373134,6.567164,2.089552,12.238806,3.880597,24.477612,17.910448,5.970149,100.0
Publication Year,53,24,19,23,7,46,14,83,64,21,...,6.779661,5.367232,6.497175,1.977401,12.994350,3.954802,23.446328,18.079096,5.932203,100.0


In [ ]:
countries_topics_list_no_fund.head(60)

Topic N°,0,1,2,3,4,5,6,7,8,9,...,1_%no_fund,2_%no_fund,3_%no_fund,4_%no_fund,5_%no_fund,6_%no_fund,7_%no_fund,8_%no_fund,9_%no_fund,no_fund_total_%no_fund
wos_ID,54,24,19,23,7,46,14,83,64,21,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
Unnamed: 0,54,24,19,23,7,46,14,83,64,21,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
funding_yes,54,24,19,23,7,46,14,83,64,21,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
open_access_yes,54,24,19,23,7,46,14,83,64,21,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
Funding Name Preferred,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
europe and central asia,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
north america,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
africa,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-saharan africa,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
east asia and pacific,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## concat all dataframe and save

In [ ]:
# concatenate dataframes on index
concat_countries_oa_list = pd.concat([countries_topics_list_all,
                                        countries_topics_list_fund,
                                        countries_topics_list_no_fund],
                                       axis=1)
# save data
concat_countries_oa_list.to_csv(f'../TABLES/concat_region_countries_org_topics_list.csv')
concat_countries_oa_list

Topic N°,0,1,2,3,4,5,6,7,8,9,...,1_%no_fund,2_%no_fund,3_%no_fund,4_%no_fund,5_%no_fund,6_%no_fund,7_%no_fund,8_%no_fund,9_%no_fund,no_fund_total_%no_fund
wos_ID,66,79,59,79,51,141,91,170,107,89,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
Unnamed: 0,66,79,59,79,51,141,91,170,107,89,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
funding_yes,66,79,59,79,51,141,91,170,107,89,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
open_access_yes,66,79,59,79,51,141,91,170,107,89,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
Funding Name Preferred,12,54,39,56,44,95,77,87,43,68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Times Cited, All Databases",56,78,55,71,49,136,86,158,96,86,...,7.407407,5.246914,6.172840,1.851852,14.197531,4.320988,23.765432,16.975309,6.481481,100.0
180 Day Usage Count,8,16,22,25,20,31,36,78,40,36,...,3.947368,3.947368,10.526316,2.631579,7.894737,3.947368,35.526316,22.368421,5.263158,100.0
Since 2013 Usage Count,60,77,58,76,50,130,87,167,103,87,...,7.164179,5.373134,6.567164,2.089552,12.238806,3.880597,24.477612,17.910448,5.970149,100.0
Publication Year,65,79,58,79,51,140,91,170,106,88,...,6.779661,5.367232,6.497175,1.977401,12.994350,3.954802,23.446328,18.079096,5.932203,100.0


## significance

In [ ]:
for topics in range(10):
    fund_topics_count = data_and_topics_number.loc[data_and_topics_number['Topic N°'] == topics]['funding_yes'].value_counts()#[1]

    print(f'Funding count Topic {topics} : 'f'{fund_topics_count}')

Funding count Topic 0 : 0    54
1    12
Name: funding_yes, dtype: int64
Funding count Topic 1 : 1    55
0    24
Name: funding_yes, dtype: int64
Funding count Topic 2 : 1    40
0    19
Name: funding_yes, dtype: int64
Funding count Topic 3 : 1    56
0    23
Name: funding_yes, dtype: int64
Funding count Topic 4 : 1    44
0     7
Name: funding_yes, dtype: int64
Funding count Topic 5 : 1    95
0    46
Name: funding_yes, dtype: int64
Funding count Topic 6 : 1    77
0    14
Name: funding_yes, dtype: int64
Funding count Topic 7 : 1    87
0    83
Name: funding_yes, dtype: int64
Funding count Topic 8 : 0    64
1    43
Name: funding_yes, dtype: int64
Funding count Topic 9 : 1    68
0    21
Name: funding_yes, dtype: int64


In [ ]:
for topics in range(10):
    fund_topics_percent = data_and_topics_number.loc[data_and_topics_number['Topic N°'] == topics]['funding_yes'].value_counts(normalize=True)#[1]

    print(f'Funding percentage Topic {topics} : 'f'{fund_topics_percent}')

Funding percentage Topic 0 : 0    0.818182
1    0.181818
Name: funding_yes, dtype: float64
Funding percentage Topic 1 : 1    0.696203
0    0.303797
Name: funding_yes, dtype: float64
Funding percentage Topic 2 : 1    0.677966
0    0.322034
Name: funding_yes, dtype: float64
Funding percentage Topic 3 : 1    0.708861
0    0.291139
Name: funding_yes, dtype: float64
Funding percentage Topic 4 : 1    0.862745
0    0.137255
Name: funding_yes, dtype: float64
Funding percentage Topic 5 : 1    0.673759
0    0.326241
Name: funding_yes, dtype: float64
Funding percentage Topic 6 : 1    0.846154
0    0.153846
Name: funding_yes, dtype: float64
Funding percentage Topic 7 : 1    0.511765
0    0.488235
Name: funding_yes, dtype: float64
Funding percentage Topic 8 : 0    0.598131
1    0.401869
Name: funding_yes, dtype: float64
Funding percentage Topic 9 : 1    0.764045
0    0.235955
Name: funding_yes, dtype: float64


In [ ]:
# for topics in range(10):


In [ ]:
data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 0][data_and_topics_number['Topic N°'] == 2]

C:\Users\dmbog\AppData\Local\Temp\ipykernel_21248\3103809225.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 0][data_and_topics_number['Topic N°'] == 2]


,Unnamed: 0,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,/,...,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year,year_group,Topic N°
wos_ID,,,,,,,,,,,,,,,,,,,,,
wos:000882496400002,12,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,gold,NaN,4.0,4.0,2022.0,2018-2023,2
wos:000405080700052,306,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"green submitted, green accepted, green publish...",2.0,NaN,2.0,2017.0,2013-2017,2
wos:000428870100012,330,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,NaN,NaN,2017.0,2013-2017,2
wos:000251090400028,693,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,bronze,35.0,1.0,3.0,2007.0,2003-2007,2
wos:000252271900001,701,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"green published, gold",22.0,NaN,6.0,2007.0,2003-2007,2
wos:000244380000030,713,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"bronze, green published, green accepted",157.0,NaN,10.0,2007.0,2003-2007,2
wos:000234382900049,739,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"green accepted, green published",78.0,NaN,2.0,2006.0,2003-2007,2
wos:000233279300042,743,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"green accepted, green published",56.0,NaN,6.0,2005.0,2003-2007,2
wos:000228236300031,753,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,bronze,58.0,NaN,8.0,2005.0,2003-2007,2


In [ ]:
fund_topics = data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 1]['Topic N°']
no_fund_topics = data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 0]['Topic N°']

In [ ]:
fund_topics = data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 1][data_and_topics_number['Topic N°'] == 0]
no_fund_topics = data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 0][data_and_topics_number['Topic N°'] != 0]

In [ ]:
test_sig(fund_topics, no_fund_topics)

mann-whitney


MannwhitneyuResult(statistic=103365.5, pvalue=0.8109456060054832)

In [ ]:
# fund_topics = data_and_topics_number.loc[(data_and_topics_number['funding_yes'] == 1) & (data_and_topics_number['Topic N°'] == 1)]['funding_yes']
# no_fund_topics = data_and_topics_number.loc[(data_and_topics_number['funding_yes'] == 0) & (data_and_topics_number['Topic N°'] == 1)]['funding_yes']

# Topics Trends year group

In [ ]:
data_and_topics_number2 = data_and_topics_number[['Publication Year','funding_yes','Topic N°','year_group']].reset_index().set_index(['wos_ID', 'Publication Year','funding_yes','year_group'])
data_and_topics_number2

,,,,Topic N°
wos_ID,Publication Year,funding_yes,year_group,
wos:000904661800001,2023.0,0,2018-2023,3
wos:000921279800001,2023.0,1,2018-2023,5
wos:000909001200001,2023.0,1,2018-2023,7
wos:000777045900001,2022.0,1,2018-2023,8
wos:000849602200001,2022.0,0,2018-2023,8
...,...,...,...,...
wos:a1968b358900049,1968.0,0,1968-1972,5
wos:a1968a855300001,1968.0,0,1968-1972,8
wos:a1968a855300006,1968.0,0,1968-1972,8


In [ ]:
data_topics_trend = pd.get_dummies(data_and_topics_number2['Topic N°'])
data_topics_trend

,,,,0,1,2,3,4,5,6,7,8,9
wos_ID,Publication Year,funding_yes,year_group,,,,,,,,,,
wos:000904661800001,2023.0,0,2018-2023,0,0,0,1,0,0,0,0,0,0
wos:000921279800001,2023.0,1,2018-2023,0,0,0,0,0,1,0,0,0,0
wos:000909001200001,2023.0,1,2018-2023,0,0,0,0,0,0,0,1,0,0
wos:000777045900001,2022.0,1,2018-2023,0,0,0,0,0,0,0,0,1,0
wos:000849602200001,2022.0,0,2018-2023,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
wos:a1968b358900049,1968.0,0,1968-1972,0,0,0,0,0,1,0,0,0,0
wos:a1968a855300001,1968.0,0,1968-1972,0,0,0,0,0,0,0,0,1,0
wos:a1968a855300006,1968.0,0,1968-1972,0,0,0,0,0,0,0,0,1,0


## ALL

In [ ]:
data_topics_trend_all = data_topics_trend.groupby('year_group').sum()

data_topics_trend_all['all_total'] = data_topics_trend_all[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_all.columns = data_topics_trend_all.columns.astype(str)

for col in data_topics_trend_all.columns:
    data_topics_trend_all[col + '_%all'] = data_topics_trend_all[col]/data_topics_trend_all['all_total']*100

data_topics_trend_all

,0,1,2,3,4,5,6,7,8,9,...,1_%all,2_%all,3_%all,4_%all,5_%all,6_%all,7_%all,8_%all,9_%all,all_total_%all
year_group,,,,,,,,,,,,,,,,,,,,,
1968-1972,3,0,0,1,0,9,0,0,5,1,...,0.000000,0.000000,5.263158,0.000000,47.368421,0.000000,0.000000,26.315789,5.263158,100.0
1973-1977,4,0,0,0,0,0,0,0,2,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,100.0
1978-1982,15,0,0,0,0,0,0,1,6,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.545455,27.272727,0.000000,100.0
1983-1987,3,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1988-1992,4,0,0,0,0,0,0,0,0,2,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,100.0
1993-1997,9,2,2,0,0,8,3,5,2,3,...,5.882353,5.882353,0.000000,0.000000,23.529412,8.823529,14.705882,5.882353,8.823529,100.0
1998-2002,8,6,8,1,2,7,5,9,6,3,...,10.909091,14.545455,1.818182,3.636364,12.727273,9.090909,16.363636,10.909091,5.454545,100.0
2003-2007,1,17,8,10,4,13,7,22,13,8,...,16.504854,7.766990,9.708738,3.883495,12.621359,6.796117,21.359223,12.621359,7.766990,100.0
2008-2012,9,20,5,17,16,29,14,28,16,17,...,11.695906,2.923977,9.941520,9.356725,16.959064,8.187135,16.374269,9.356725,9.941520,100.0


## fund

In [ ]:
data_fund = data_topics_trend.reset_index().loc[data_topics_trend.reset_index()['funding_yes']==1].set_index(['wos_ID','funding_yes', 'Publication Year'])
data_fund.head(2)

,,,year_group,0,1,2,3,4,5,6,7,8,9
wos_ID,funding_yes,Publication Year,,,,,,,,,,,
wos:000921279800001,1,2023.0,2018-2023,0,0,0,0,0,1,0,0,0,0
wos:000909001200001,1,2023.0,2018-2023,0,0,0,0,0,0,0,1,0,0


In [ ]:
data_topics_trend_fund = data_fund.groupby('year_group').sum()

data_topics_trend_fund['fund_total'] = data_topics_trend_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_fund.columns = data_topics_trend_fund.columns.astype(str)

for col in data_topics_trend_fund.columns:
    data_topics_trend_fund[col + '_%fund'] = data_topics_trend_fund[col]/data_topics_trend_fund['fund_total']*100


data_topics_trend_fund

,0,1,2,3,4,5,6,7,8,9,...,1_%fund,2_%fund,3_%fund,4_%fund,5_%fund,6_%fund,7_%fund,8_%fund,9_%fund,fund_total_%fund
year_group,,,,,,,,,,,,,,,,,,,,,
1998-2002,0,0,0,0,0,0,1,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,100.0
2003-2007,0,3,2,0,2,5,3,1,0,3,...,15.789474,10.526316,0.000000,10.526316,26.315789,15.789474,5.263158,0.000000,15.789474,100.0
2008-2012,6,19,5,12,16,20,13,11,9,15,...,15.079365,3.968254,9.523810,12.698413,15.873016,10.317460,8.730159,7.142857,11.904762,100.0
2013-2017,5,18,16,17,15,26,23,33,13,24,...,9.473684,8.421053,8.947368,7.894737,13.684211,12.105263,17.368421,6.842105,12.631579,100.0
2018-2023,1,15,16,27,11,43,37,42,20,25,...,6.329114,6.751055,11.392405,4.641350,18.143460,15.611814,17.721519,8.438819,10.548523,100.0


## no fund

In [ ]:
data_no_fund = data_topics_trend.reset_index().loc[data_topics_trend.reset_index()['funding_yes']==0].set_index(['wos_ID','funding_yes', 'Publication Year'])
data_no_fund.head(2)

,,,year_group,0,1,2,3,4,5,6,7,8,9
wos_ID,funding_yes,Publication Year,,,,,,,,,,,
wos:000904661800001,0,2023.0,2018-2023,0,0,0,1,0,0,0,0,0,0
wos:000849602200001,0,2022.0,2018-2023,0,0,0,0,0,0,0,0,1,0


In [ ]:
data_topics_trend_no_fund = data_no_fund.groupby('year_group').sum()

data_topics_trend_no_fund['no_fund_total'] = data_topics_trend_no_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_no_fund.columns = data_topics_trend_no_fund.columns.astype(str)

for col in data_topics_trend_no_fund.columns:
    data_topics_trend_no_fund[col + '_%no_fund'] = data_topics_trend_no_fund[col]/data_topics_trend_no_fund['no_fund_total']*100


data_topics_trend_no_fund

,0,1,2,3,4,5,6,7,8,9,...,1_%no_fund,2_%no_fund,3_%no_fund,4_%no_fund,5_%no_fund,6_%no_fund,7_%no_fund,8_%no_fund,9_%no_fund,no_fund_total_%no_fund
year_group,,,,,,,,,,,,,,,,,,,,,
1968-1972,3,0,0,1,0,9,0,0,5,1,...,0.000000,0.000000,5.263158,0.000000,47.368421,0.000000,0.000000,26.315789,5.263158,100.0
1973-1977,4,0,0,0,0,0,0,0,2,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,100.0
1978-1982,15,0,0,0,0,0,0,1,6,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.545455,27.272727,0.000000,100.0
1983-1987,3,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1988-1992,4,0,0,0,0,0,0,0,0,2,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,100.0
1993-1997,9,2,2,0,0,8,3,5,2,3,...,5.882353,5.882353,0.000000,0.000000,23.529412,8.823529,14.705882,5.882353,8.823529,100.0
1998-2002,8,6,8,1,2,7,4,9,6,3,...,11.111111,14.814815,1.851852,3.703704,12.962963,7.407407,16.666667,11.111111,5.555556,100.0
2003-2007,1,14,6,10,2,8,4,21,13,5,...,16.666667,7.142857,11.904762,2.380952,9.523810,4.761905,25.000000,15.476190,5.952381,100.0
2008-2012,3,1,0,5,0,9,1,17,7,2,...,2.222222,0.000000,11.111111,0.000000,20.000000,2.222222,37.777778,15.555556,4.444444,100.0


## concat and save

In [ ]:
# concatenate dataframes on index
concat_data_topics_trend = pd.concat([data_topics_trend_all,
                                    data_topics_trend_fund,
                                    data_topics_trend_no_fund],
                                    axis=1)
# save data
concat_data_topics_trend.to_csv(f'../TABLES/concat_data_topics_trend_year_group.csv')
concat_data_topics_trend

,0,1,2,3,4,5,6,7,8,9,...,1_%no_fund,2_%no_fund,3_%no_fund,4_%no_fund,5_%no_fund,6_%no_fund,7_%no_fund,8_%no_fund,9_%no_fund,no_fund_total_%no_fund
year_group,,,,,,,,,,,,,,,,,,,,,
1968-1972,3,0,0,1,0,9,0,0,5,1,...,0.000000,0.000000,5.263158,0.000000,47.368421,0.000000,0.000000,26.315789,5.263158,100.0
1973-1977,4,0,0,0,0,0,0,0,2,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,100.0
1978-1982,15,0,0,0,0,0,0,1,6,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.545455,27.272727,0.000000,100.0
1983-1987,3,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1988-1992,4,0,0,0,0,0,0,0,0,2,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,100.0
1993-1997,9,2,2,0,0,8,3,5,2,3,...,5.882353,5.882353,0.000000,0.000000,23.529412,8.823529,14.705882,5.882353,8.823529,100.0
1998-2002,8,6,8,1,2,7,5,9,6,3,...,11.111111,14.814815,1.851852,3.703704,12.962963,7.407407,16.666667,11.111111,5.555556,100.0
2003-2007,1,17,8,10,4,13,7,22,13,8,...,16.666667,7.142857,11.904762,2.380952,9.523810,4.761905,25.000000,15.476190,5.952381,100.0
2008-2012,9,20,5,17,16,29,14,28,16,17,...,2.222222,0.000000,11.111111,0.000000,20.000000,2.222222,37.777778,15.555556,4.444444,100.0


# Topics Trends

In [ ]:
data_and_topics_number2 = data_and_topics_number[['Publication Year','funding_yes','Topic N°','year_group']].reset_index().set_index(['wos_ID', 'Publication Year','funding_yes','year_group'])
data_and_topics_number2

,,,,Topic N°
wos_ID,Publication Year,funding_yes,year_group,
wos:000904661800001,2023.0,0,2018-2023,3
wos:000921279800001,2023.0,1,2018-2023,5
wos:000909001200001,2023.0,1,2018-2023,7
wos:000777045900001,2022.0,1,2018-2023,8
wos:000849602200001,2022.0,0,2018-2023,8
...,...,...,...,...
wos:a1968b358900049,1968.0,0,1968-1972,5
wos:a1968a855300001,1968.0,0,1968-1972,8
wos:a1968a855300006,1968.0,0,1968-1972,8


In [ ]:
data_topics_trend = pd.get_dummies(data_and_topics_number2['Topic N°'])
data_topics_trend

,,,,0,1,2,3,4,5,6,7,8,9
wos_ID,Publication Year,funding_yes,year_group,,,,,,,,,,
wos:000904661800001,2023.0,0,2018-2023,0,0,0,1,0,0,0,0,0,0
wos:000921279800001,2023.0,1,2018-2023,0,0,0,0,0,1,0,0,0,0
wos:000909001200001,2023.0,1,2018-2023,0,0,0,0,0,0,0,1,0,0
wos:000777045900001,2022.0,1,2018-2023,0,0,0,0,0,0,0,0,1,0
wos:000849602200001,2022.0,0,2018-2023,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
wos:a1968b358900049,1968.0,0,1968-1972,0,0,0,0,0,1,0,0,0,0
wos:a1968a855300001,1968.0,0,1968-1972,0,0,0,0,0,0,0,0,1,0
wos:a1968a855300006,1968.0,0,1968-1972,0,0,0,0,0,0,0,0,1,0


## ALL

In [ ]:
data_topics_trend_all = data_topics_trend.groupby('Publication Year').sum()

data_topics_trend_all['all_total'] = data_topics_trend_all[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_all.columns = data_topics_trend_all.columns.astype(str)

for col in data_topics_trend_all.columns:
    data_topics_trend_all[col + '_%all'] = data_topics_trend_all[col]/data_topics_trend_all['all_total']*100

data_topics_trend_all

,0,1,2,3,4,5,6,7,8,9,...,1_%all,2_%all,3_%all,4_%all,5_%all,6_%all,7_%all,8_%all,9_%all,all_total_%all
Publication Year,,,,,,,,,,,,,,,,,,,,,
1968.0,1,0,0,0,0,5,0,0,4,0,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,40.000000,0.000000,100.0
1969.0,0,0,0,1,0,2,0,0,1,0,...,0.000000,0.000000,25.000000,0.000000,50.000000,0.000000,0.000000,25.000000,0.000000,100.0
1970.0,2,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1971.0,0,0,0,0,0,1,0,0,0,1,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000,50.000000,100.0
1972.0,0,0,0,0,0,1,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,100.0
1973.0,1,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1974.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1975.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1977.0,3,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0


## fund

In [ ]:
data_fund = data_topics_trend.reset_index().loc[data_topics_trend.reset_index()['funding_yes']==1].set_index(['wos_ID','funding_yes'])
data_fund.head(2)

,,Publication Year,year_group,0,1,2,3,4,5,6,7,8,9
wos_ID,funding_yes,,,,,,,,,,,,
wos:000921279800001,1,2023.0,2018-2023,0,0,0,0,0,1,0,0,0,0
wos:000909001200001,1,2023.0,2018-2023,0,0,0,0,0,0,0,1,0,0


In [ ]:
data_topics_trend_fund = data_fund.groupby('Publication Year').sum()

data_topics_trend_fund['fund_total'] = data_topics_trend_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_fund.columns = data_topics_trend_fund.columns.astype(str)

for col in data_topics_trend_fund.columns:
    data_topics_trend_fund[col + '_%fund'] = data_topics_trend_fund[col]/data_topics_trend_fund['fund_total']*100


data_topics_trend_fund

,0,1,2,3,4,5,6,7,8,9,...,1_%fund,2_%fund,3_%fund,4_%fund,5_%fund,6_%fund,7_%fund,8_%fund,9_%fund,fund_total_%fund
Publication Year,,,,,,,,,,,,,,,,,,,,,
2002.0,0,0,0,0,0,0,1,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,100.0
2003.0,0,1,0,0,0,0,0,0,0,0,...,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
2004.0,0,0,0,0,0,0,0,1,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,100.0
2005.0,0,0,0,0,0,0,2,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,100.0
2006.0,0,0,0,0,2,3,0,0,0,0,...,0.000000,0.000000,0.000000,40.000000,60.000000,0.000000,0.000000,0.000000,0.000000,100.0
2007.0,0,2,2,0,0,2,1,0,0,3,...,20.000000,20.000000,0.000000,0.000000,20.000000,10.000000,0.000000,0.000000,30.000000,100.0
2008.0,0,0,1,3,0,2,0,1,0,0,...,0.000000,14.285714,42.857143,0.000000,28.571429,0.000000,14.285714,0.000000,0.000000,100.0
2009.0,2,4,1,1,1,2,0,2,2,2,...,23.529412,5.882353,5.882353,5.882353,11.764706,0.000000,11.764706,11.764706,11.764706,100.0
2010.0,0,7,1,1,3,4,2,0,2,5,...,28.000000,4.000000,4.000000,12.000000,16.000000,8.000000,0.000000,8.000000,20.000000,100.0


## no fund

In [ ]:
data_no_fund = data_topics_trend.reset_index().loc[data_topics_trend.reset_index()['funding_yes']==0].set_index(['wos_ID','funding_yes'])
data_no_fund.head(2)

,,Publication Year,year_group,0,1,2,3,4,5,6,7,8,9
wos_ID,funding_yes,,,,,,,,,,,,
wos:000904661800001,0,2023.0,2018-2023,0,0,0,1,0,0,0,0,0,0
wos:000849602200001,0,2022.0,2018-2023,0,0,0,0,0,0,0,0,1,0


In [ ]:
data_topics_trend_no_fund = data_no_fund.groupby('Publication Year').sum()

data_topics_trend_no_fund['no_fund_total'] = data_topics_trend_no_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_no_fund.columns = data_topics_trend_no_fund.columns.astype(str)

for col in data_topics_trend_no_fund.columns:
    data_topics_trend_no_fund[col + '_%no_fund'] = data_topics_trend_no_fund[col]/data_topics_trend_no_fund['no_fund_total']*100


data_topics_trend_no_fund

,0,1,2,3,4,5,6,7,8,9,...,1_%no_fund,2_%no_fund,3_%no_fund,4_%no_fund,5_%no_fund,6_%no_fund,7_%no_fund,8_%no_fund,9_%no_fund,no_fund_total_%no_fund
Publication Year,,,,,,,,,,,,,,,,,,,,,
1968.0,1,0,0,0,0,5,0,0,4,0,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,40.000000,0.000000,100.0
1969.0,0,0,0,1,0,2,0,0,1,0,...,0.000000,0.000000,25.000000,0.000000,50.000000,0.000000,0.000000,25.000000,0.000000,100.0
1970.0,2,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1971.0,0,0,0,0,0,1,0,0,0,1,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000,50.000000,100.0
1972.0,0,0,0,0,0,1,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,100.0
1973.0,1,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1974.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1975.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1977.0,3,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0


## concat and save

In [ ]:
# concatenate dataframes on index
concat_data_topics_trend = pd.concat([data_topics_trend_all,
                                    data_topics_trend_fund,
                                    data_topics_trend_no_fund],
                                    axis=1)
# save data
concat_data_topics_trend.to_csv(f'../TABLES/concat_data_topics_trend_year.csv')
concat_data_topics_trend

,0,1,2,3,4,5,6,7,8,9,...,1_%no_fund,2_%no_fund,3_%no_fund,4_%no_fund,5_%no_fund,6_%no_fund,7_%no_fund,8_%no_fund,9_%no_fund,no_fund_total_%no_fund
Publication Year,,,,,,,,,,,,,,,,,,,,,
1968.0,1,0,0,0,0,5,0,0,4,0,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,40.000000,0.000000,100.0
1969.0,0,0,0,1,0,2,0,0,1,0,...,0.000000,0.000000,25.000000,0.000000,50.000000,0.000000,0.000000,25.000000,0.000000,100.0
1970.0,2,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1971.0,0,0,0,0,0,1,0,0,0,1,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000,50.000000,100.0
1972.0,0,0,0,0,0,1,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,100.0
1973.0,1,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1974.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1975.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1977.0,3,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
